In [1]:
import pandas as pd
import os
import gzip
import numpy as np
import json
import ast
import geopandas
from geopandas import GeoDataFrame
from shapely.geometry import Point, Polygon, MultiPolygon
from haversine import haversine, Unit
from shapely import wkt
import warnings
warnings.filterwarnings("ignore")
from sklearn.metrics.pairwise import cosine_similarity

year = 2018

In [2]:
answers = {'cbg': [], 'safegraph_place_id': []}

# A-D

In [3]:
with open('F:\\safegraph\\nyc_cbgs.json') as f:
    data = json.load(f)
nyc_list = data['features']
nyc_cbg = []
for i in nyc_list:
    nyc_cbg.append(i['properties']['GEOID'])

In [4]:
def get_stores_foot(group, y):
    if group == 'Target':
        walk_path_filtered = r'F:\safegraph\matrix_0920_Target'
    else:
        walk_path_filtered = r'F:\safegraph\competitors_1030\matrix'
    datapath_all_filtered = os.listdir(walk_path_filtered)
    datapath_all_filtered = [walk_path_filtered +'\\'+i for i in datapath_all_filtered]
    todo_list = [i for i in datapath_all_filtered if str(y) in i]
    if group == 'Target':
        data_filtered = [pd.read_csv(i).rename(columns = {'Unnamed: 0.1': 'cbg'}).set_index('cbg').drop(columns = ['Unnamed: 0']) 
                         for i in todo_list]
    else:
        data_filtered = [pd.read_csv(i).rename(columns = {'Unnamed: 0': 'cbg'}). set_index('cbg') for i in todo_list]
    final_data = sum(data_filtered).reset_index()
    return final_data

In [5]:
target_visitors = get_stores_foot('Target', year)
competitors_visitors = get_stores_foot('c', year)
visitors = pd.merge(target_visitors, competitors_visitors, on='cbg')

In [6]:
ids = visitors.columns[1:].tolist()

In [7]:
visitors

,cbg,sg:51c8d943ff72400ab86e7e8896f4b5af,sg:f189f91419974d03a20257132d73da58,sg:c7222887b3834a58a6bc40c168cdb38a,sg:e2543bebe82742a1941c21660d9f4168,sg:3275834c58d241baacdb9965f5003af6,sg:81aae721ea114787a6e105b349eeb400,sg:cffe77a3b2f543af9387cdc0a69cb0b8,sg:4c2a6e828527407d863fc641eaa20ac3,sg:084ab2c726784ab490b72144d44d4ee8,...,sg:a61e1c7be39f4294824e1bd68483fb03,sg:0031a04b56d64aecab115f342a98234b,sg:7f1fde9fd68b47899bfab9cd5f0071df,sg:8d376e7211074b6f8da481353e0a4627,sg:0d9fe727bf634df3b1f5c9f7460f75e4,sg:29b14793c6a54fe69540bddefc24042e,sg:187d4b6602c845c3bfede20a1deeb8d1,sg:90a04f3d29784b84ad73e55909e864f8,sg:10efeb7c06ac4940a3d4e996c0ce5476,sg:c38f81e7fe2b4b48a6e258824983b741
0,360810140001,0,12,0,0,0,0,0,0,0,...,20,4,0,0,0,0,0,0,0,0
1,360810141001,0,0,0,0,0,0,0,0,0,...,4,0,0,0,0,0,0,0,0,0
2,360810142012,0,4,0,0,0,0,0,0,4,...,12,8,0,0,0,0,0,0,0,0
3,360810142011,0,4,0,0,0,0,0,0,0,...,0,12,0,0,0,0,0,0,0,0
4,360810142021,0,4,0,0,0,0,0,0,0,...,8,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6488,360470271002,0,29,0,0,0,0,0,0,4,...,4,12,0,0,0,0,0,0,0,0
6489,360470280002,0,17,0,0,0,0,0,36,4,...,0,4,0,0,0,0,0,0,0,0
6490,360470282002,4,9,0,4,0,0,5,8,0,...,0,0,0,0,0,0,0,0,0,0
6491,360470282003,0,9,4,0,0,0,0,4,0,...,0,0,0,0,0,0,0,0,0,0


In [8]:
visitors = visitors.set_index('cbg')
visitors_sum = visitors.sum(axis = 1)
visitors_percentage = visitors.div(visitors_sum, axis='rows')
visitors = visitors.reset_index()
visitors_percentage = visitors_percentage.reset_index()

In [9]:
cbgs = visitors.cbg.tolist()

In [10]:
def turn_to_dict_visitors(df):
    dict_temp = {}
    for i,cont in df.iterrows():
        dict_temp[cont.cbg] = {}
        for store_id in ids:
            dict_temp[cont.cbg][store_id] = cont[store_id]
    return dict_temp

In [11]:
visitors_dict = turn_to_dict_visitors(visitors)

In [12]:
for c in cbgs:
    for i in ids:
        answers['cbg'].append(c)
        answers['safegraph_place_id'].append(i)
answers_df = pd.DataFrame(answers)

In [13]:
def get_visitors_data(dict_here, row):
    return dict_here[row.cbg][row.safegraph_place_id]

answers_df['Number_of_Visits_'+str(year)] = answers_df.apply(lambda row: get_visitors_data(visitors_dict, row), axis=1)

In [14]:
answers_df

,cbg,safegraph_place_id,Number_of_Visits_2018
0,360810140001,sg:51c8d943ff72400ab86e7e8896f4b5af,0
1,360810140001,sg:f189f91419974d03a20257132d73da58,12
2,360810140001,sg:c7222887b3834a58a6bc40c168cdb38a,0
3,360810140001,sg:e2543bebe82742a1941c21660d9f4168,0
4,360810140001,sg:3275834c58d241baacdb9965f5003af6,0
...,...,...,...
1844007,360470286004,sg:29b14793c6a54fe69540bddefc24042e,0
1844008,360470286004,sg:187d4b6602c845c3bfede20a1deeb8d1,0
1844009,360470286004,sg:90a04f3d29784b84ad73e55909e864f8,0
1844010,360470286004,sg:10efeb7c06ac4940a3d4e996c0ce5476,0


# E and N

In [15]:
with open('F:\\safegraph\\nyc_cbgs.json') as f:
    cbg_data = json.load(f)
cbg_info_full = cbg_data['features']

In [16]:
shp_path = 'F:\\safegraph\\tl_2017_36_bg\\tl_2017_36_bg.shp'
shp_df = geopandas.GeoDataFrame.from_file(shp_path,encoding = 'gb18030')
cbg_info = {}
cbg_id = shp_df['GEOID'].tolist()
cbg_geo = shp_df['geometry'].tolist()
for i in range(len(cbg_id)):
    if int(cbg_id[i]) in cbgs:
        cbg_info[cbg_id[i]] = cbg_geo[i]

In [17]:
census_data = pd.read_csv(r'F:\safegraph\census data\census_data_by_cbg.csv').drop(columns  = 'Unnamed: 0')
census_data['cbg_geometry'] = census_data['cbg_geometry'].apply(wkt.loads)
census_gdf = geopandas.GeoDataFrame(census_data, geometry = 'cbg_geometry')

In [18]:
census_gdf

,census_block_group,population,cbg_geometry,median_income
0,360050001000,0,"POLYGON ((-73.89772 40.79514, -73.89611 40.796...",0.0
1,360050001001,0,"POLYGON ((-73.89266 40.79382, -73.89194 40.796...",0.0
2,360050002000,0,"POLYGON ((-73.86648 40.80590, -73.86231 40.809...",0.0
3,360050002001,2109,"POLYGON ((-73.86333 40.81478, -73.86240 40.814...",62375.0
4,360050002002,2168,"POLYGON ((-73.86522 40.81453, -73.86425 40.814...",75000.0
...,...,...,...,...
6488,360850319022,1277,"POLYGON ((-74.17122 40.63744, -74.16969 40.637...",64118.0
6489,360850319023,1308,"POLYGON ((-74.16619 40.63585, -74.16509 40.635...",0.0
6490,360850323000,0,"POLYGON ((-74.19709 40.63574, -74.19706 40.635...",0.0
6491,360850323001,1218,"POLYGON ((-74.19592 40.63522, -74.19589 40.635...",64250.0


In [19]:
cbg_pop_dict = {}
cbg_mincome_dict = {}
for i,cont in census_gdf.iterrows():
    cbg_pop_dict[cont.census_block_group] = cont.population
    cbg_mincome_dict[cont.census_block_group] = cont.median_income

In [20]:
answers_df['Population_in_this_cbg'] = answers_df['cbg'].apply(lambda x: cbg_pop_dict[x])
answers_df['Median_Income_in_this_cbg'] = answers_df['cbg'].apply(lambda x: cbg_mincome_dict[x])

In [21]:
answers_df

,cbg,safegraph_place_id,Number_of_Visits_2018,Population_in_this_cbg,Median_Income_in_this_cbg
0,360810140001,sg:51c8d943ff72400ab86e7e8896f4b5af,0,2187,69750.0
1,360810140001,sg:f189f91419974d03a20257132d73da58,12,2187,69750.0
2,360810140001,sg:c7222887b3834a58a6bc40c168cdb38a,0,2187,69750.0
3,360810140001,sg:e2543bebe82742a1941c21660d9f4168,0,2187,69750.0
4,360810140001,sg:3275834c58d241baacdb9965f5003af6,0,2187,69750.0
...,...,...,...,...,...
1844007,360470286004,sg:29b14793c6a54fe69540bddefc24042e,0,1147,59306.0
1844008,360470286004,sg:187d4b6602c845c3bfede20a1deeb8d1,0,1147,59306.0
1844009,360470286004,sg:90a04f3d29784b84ad73e55909e864f8,0,1147,59306.0
1844010,360470286004,sg:10efeb7c06ac4940a3d4e996c0ce5476,0,1147,59306.0


# F and H

In [22]:
poi_nyc = pd.read_csv('F:\safegraph\sample_data\core_USA\poi_NYC.csv')
square_data = pd.read_csv(r'F:\safegraph\sample_data\SafeGraphPlacesGeoSupplementSquareFeet.csv')

In [23]:
stores_with_poi_data = poi_nyc[poi_nyc['safegraph_place_id'].isin(visitors.columns.tolist())][['safegraph_place_id', 'longitude', 'latitude']]
square_data = square_data[square_data['safegraph_place_id'].isin(visitors.columns.tolist())][['safegraph_place_id', 'area_square_feet']]
geo_area_stores = pd.merge(stores_with_poi_data, square_data, on = 'safegraph_place_id')

In [24]:
geo_area_stores = geo_area_stores.append({'safegraph_place_id':'aggregated_517E','longitude': -73.93100799999999, 'latitude': 40.795408,
                                          'area_square_feet':169356}, ignore_index=True)
geo_area_stores = geo_area_stores.append({'safegraph_place_id':'aggregated_4024_College','longitude': -73.834744, 'latitude': 40.757749,
                                          'area_square_feet':214735}, ignore_index=True)
geo_area_stores = geo_area_stores.append({'safegraph_place_id':'aggregated_40W','longitude': -73.907558, 'latitude': 40.872776, 
                                          'area_square_feet':183593}, ignore_index=True)

In [25]:
def get_cbg(p1):
    for k,v in cbg_info.items():
        if p1.within(v):
            return k
    return 'No'

In [26]:
geometry_poi = [Point(xy) for xy in zip(geo_area_stores.longitude, geo_area_stores.latitude)]
geo_area_stores_gdf = GeoDataFrame(geo_area_stores, crs="EPSG:4326", geometry=geometry_poi)
geo_area_stores_gdf['geo'] = geo_area_stores_gdf['geometry'].apply(lambda x: (x.y, x.x))
geo_area_stores_gdf['cbg'] = geo_area_stores['geometry'].apply(lambda x: get_cbg(x))
geo_area_stores_gdf = geo_area_stores_gdf[geo_area_stores_gdf['cbg'] != 'No']
geo_area_stores_gdf['cbg'] = geo_area_stores_gdf['cbg'].astype('int64')

In [27]:
final_stores = geo_area_stores_gdf['safegraph_place_id'].tolist()

In [28]:
geo_area_stores_gdf['cbg_population'] = geo_area_stores_gdf['cbg'].apply(lambda x: cbg_pop_dict[x])
geo_area_stores_gdf['cbg_median_income'] = geo_area_stores_gdf['cbg'].apply(lambda x: cbg_mincome_dict[int(x)])

In [29]:
store_pop_dict = {}
store_mincome_dict = {}
store_area_dict = {}
for i, cont in geo_area_stores_gdf.iterrows():
    store_pop_dict[cont.safegraph_place_id] = cont.cbg_population
    store_mincome_dict[cont.safegraph_place_id] = cont.cbg_median_income
    store_area_dict[cont.safegraph_place_id] = cont.area_square_feet

In [30]:
geo_area_stores_gdf

,safegraph_place_id,longitude,latitude,area_square_feet,geometry,geo,cbg,cbg_population,cbg_median_income
0,sg:1b84da99a0a545c99110a376a840293c,-74.191397,40.600690,22192,POINT (-74.19140 40.60069),"(40.60069, -74.191397)",360850291021,922,76250.0
1,sg:6eb37e997f3e4ea68c4ffda26c95ebf2,-73.811354,40.587022,11080,POINT (-73.81135 40.58702),"(40.587022, -73.81135400000002)",360810942032,1040,50667.0
2,sg:670f3d81cc5f44f5835954f0285f57af,-74.073002,40.617067,4740,POINT (-74.07300 40.61707),"(40.617067, -74.073002)",360850008001,1407,68259.0
3,sg:a1eb0e6f38994e40859b4bf5e9fc15d4,-74.156726,40.624749,13175,POINT (-74.15673 40.62475),"(40.624749, -74.156726)",360850303024,409,102656.0
4,sg:0d9fe727bf634df3b1f5c9f7460f75e4,-73.916358,40.806030,18404,POINT (-73.91636 40.80603),"(40.80603, -73.916358)",360050027012,1670,23066.0
...,...,...,...,...,...,...,...,...,...
279,sg:84a8615ac25b455bb1d01ec4c93cdd1f,-74.165371,40.583277,96668,POINT (-74.16537 40.58328),"(40.583277, -74.165371)",360850277022,1848,60278.0
280,sg:ede0c3b8b3a143848181ee2430390072,-73.908321,40.659248,5885,POINT (-73.90832 40.65925),"(40.659248, -73.908321)",360470916003,1206,39036.0
281,aggregated_517E,-73.931008,40.795408,169356,POINT (-73.93101 40.79541),"(40.795408, -73.93100799999999)",360610178003,945,54071.0
282,aggregated_4024_College,-73.834744,40.757749,214735,POINT (-73.83474 40.75775),"(40.757749, -73.834744)",360810871001,1781,32500.0


In [31]:
answers_df = answers_df[answers_df['safegraph_place_id'].isin(final_stores)]

In [32]:
answers_df['Population_where_store_is'] = answers_df['safegraph_place_id'].apply(lambda x: store_pop_dict[x])
answers_df['Median_Income_where_store_is'] = answers_df['safegraph_place_id'].apply(lambda x: store_mincome_dict[x])

In [33]:
answers_df['Area_of_store'] = answers_df['safegraph_place_id'].apply(lambda x: store_area_dict[x])
answers_df

,cbg,safegraph_place_id,Number_of_Visits_2018,Population_in_this_cbg,Median_Income_in_this_cbg,Population_where_store_is,Median_Income_where_store_is,Area_of_store
0,360810140001,sg:51c8d943ff72400ab86e7e8896f4b5af,0,2187,69750.0,287,36006.0,135702
1,360810140001,sg:f189f91419974d03a20257132d73da58,12,2187,69750.0,1348,80663.0,149385
2,360810140001,sg:c7222887b3834a58a6bc40c168cdb38a,0,2187,69750.0,1280,0.0,3678
3,360810140001,sg:e2543bebe82742a1941c21660d9f4168,0,2187,69750.0,1796,75547.0,30554
4,360810140001,sg:3275834c58d241baacdb9965f5003af6,0,2187,69750.0,163,0.0,6135
...,...,...,...,...,...,...,...,...
1844007,360470286004,sg:29b14793c6a54fe69540bddefc24042e,0,1147,59306.0,1823,26917.0,2237
1844008,360470286004,sg:187d4b6602c845c3bfede20a1deeb8d1,0,1147,59306.0,1464,30817.0,13671
1844009,360470286004,sg:90a04f3d29784b84ad73e55909e864f8,0,1147,59306.0,1348,80663.0,25223
1844010,360470286004,sg:10efeb7c06ac4940a3d4e996c0ce5476,0,1147,59306.0,1643,29944.0,28716


# G

In [34]:
with open('F:\\safegraph\\nyc_cbgs.json') as f:
    cbg_data = json.load(f)
cbg_info = cbg_data['features']

In [35]:
def get_geo(cbg):
    for c in cbg_info_full:
        if c['properties']['GEOID'] == str(cbg):
            lat,long = c['properties']['INTPTLAT'], c['properties']['INTPTLON']
            return float(lat), float(long)

In [36]:
store_geo_dict = {}
for i,cont in geo_area_stores_gdf.iterrows():
    store_geo_dict[cont.safegraph_place_id] = {}
    store_geo_dict[cont.safegraph_place_id]['geometry'] = cont.geometry
    store_geo_dict[cont.safegraph_place_id]['geo'] = cont.geo

In [37]:
def get_haversine(row):
    geo_cbg = get_geo(str(row.cbg))
    store_location = store_geo_dict[row.safegraph_place_id]['geo']
    return haversine(store_location, geo_cbg, unit = Unit.MILES)

In [38]:
answers_df['Distance_between_cbg_and_store'] = answers_df.apply(lambda row: get_haversine(row), axis=1)

In [39]:
answers_df

,cbg,safegraph_place_id,Number_of_Visits_2018,Population_in_this_cbg,Median_Income_in_this_cbg,Population_where_store_is,Median_Income_where_store_is,Area_of_store,Distance_between_cbg_and_store
0,360810140001,sg:51c8d943ff72400ab86e7e8896f4b5af,0,2187,69750.0,287,36006.0,135702,9.780264
1,360810140001,sg:f189f91419974d03a20257132d73da58,12,2187,69750.0,1348,80663.0,149385,4.111893
2,360810140001,sg:c7222887b3834a58a6bc40c168cdb38a,0,2187,69750.0,1280,0.0,3678,8.292913
3,360810140001,sg:e2543bebe82742a1941c21660d9f4168,0,2187,69750.0,1796,75547.0,30554,8.162305
4,360810140001,sg:3275834c58d241baacdb9965f5003af6,0,2187,69750.0,163,0.0,6135,8.393494
...,...,...,...,...,...,...,...,...,...
1844007,360470286004,sg:29b14793c6a54fe69540bddefc24042e,0,1147,59306.0,1823,26917.0,2237,18.623994
1844008,360470286004,sg:187d4b6602c845c3bfede20a1deeb8d1,0,1147,59306.0,1464,30817.0,13671,6.841547
1844009,360470286004,sg:90a04f3d29784b84ad73e55909e864f8,0,1147,59306.0,1348,80663.0,25223,7.679363
1844010,360470286004,sg:10efeb7c06ac4940a3d4e996c0ce5476,0,1147,59306.0,1643,29944.0,28716,16.034442


# I

In [40]:
parking_1 = pd.read_csv(r'F:\safegraph\presentation\NY-MA10-15-2020-19-16-CORE_POI-GEOMETRY-2020_09-2020-10-16\core_poi-geometry.csv\core_poi-geometry.csv')
parking_2 = pd.read_csv(r'F:\safegraph\presentation\interactive_map\first_step_data_with_competitors.csv')

In [41]:
parking_dict = {}
for i,cont in parking_1.iterrows():
    if cont.safegraph_place_id in final_stores:
        if cont.includes_parking_lot == 'False':
            parking_dict[cont.safegraph_place_id] = 0
        else:
            parking_dict[cont.safegraph_place_id] = 1
for i,cont in parking_2.iterrows():
    if cont.safegraph_place_id in final_stores:
        parking_dict[cont.safegraph_place_id] = cont.includes_parking_lot

for s in final_stores:
    if s not in parking_dict.keys():
        parking_dict[s] = 0.0001

In [42]:
answers_df['Includes_parking_lot'] = answers_df['safegraph_place_id'].apply(lambda x: parking_dict[x])

In [43]:
answers_df

,cbg,safegraph_place_id,Number_of_Visits_2018,Population_in_this_cbg,Median_Income_in_this_cbg,Population_where_store_is,Median_Income_where_store_is,Area_of_store,Distance_between_cbg_and_store,Includes_parking_lot
0,360810140001,sg:51c8d943ff72400ab86e7e8896f4b5af,0,2187,69750.0,287,36006.0,135702,9.780264,0.0000
1,360810140001,sg:f189f91419974d03a20257132d73da58,12,2187,69750.0,1348,80663.0,149385,4.111893,1.0000
2,360810140001,sg:c7222887b3834a58a6bc40c168cdb38a,0,2187,69750.0,1280,0.0,3678,8.292913,0.0000
3,360810140001,sg:e2543bebe82742a1941c21660d9f4168,0,2187,69750.0,1796,75547.0,30554,8.162305,0.0000
4,360810140001,sg:3275834c58d241baacdb9965f5003af6,0,2187,69750.0,163,0.0,6135,8.393494,1.0000
...,...,...,...,...,...,...,...,...,...,...
1844007,360470286004,sg:29b14793c6a54fe69540bddefc24042e,0,1147,59306.0,1823,26917.0,2237,18.623994,1.0000
1844008,360470286004,sg:187d4b6602c845c3bfede20a1deeb8d1,0,1147,59306.0,1464,30817.0,13671,6.841547,0.0001
1844009,360470286004,sg:90a04f3d29784b84ad73e55909e864f8,0,1147,59306.0,1348,80663.0,25223,7.679363,0.0001
1844010,360470286004,sg:10efeb7c06ac4940a3d4e996c0ce5476,0,1147,59306.0,1643,29944.0,28716,16.034442,0.0001


# K and L

In [44]:
poi_diversity = pd.read_csv(r'F:\safegraph\POI\poi_diversity_count_per_cbg.csv')

In [45]:
poi_diversity

,Unnamed: 0,cbg,diversity,Number
0,0,360810942032,2.428274,18
1,1,360471134003,0.636514,3
2,2,360470533005,1.945910,12
3,3,360810061002,1.810161,25
4,4,360610063005,2.415348,35
...,...,...,...,...
5976,5976,360050070002,1.515708,11
5977,5977,360610159002,1.549826,7
5978,5978,360470329003,2.484907,12
5979,5979,360050222001,1.979562,21


In [46]:
poi_with_stores = pd.merge(poi_diversity, geo_area_stores_gdf[['cbg', 'safegraph_place_id']], on = 'cbg')

In [47]:
store_count_dict = {}
store_diversity_dict = {}
for i,cont in poi_with_stores.iterrows():
    store_count_dict[cont.safegraph_place_id] = cont.Number
    store_diversity_dict[cont.safegraph_place_id] = cont.diversity

In [48]:
answers_df['POI_count_where_store_is'] = answers_df['safegraph_place_id'].apply(lambda x: store_count_dict[x])
answers_df['POI_diversity_where_store_is'] = answers_df['safegraph_place_id'].apply(lambda x: store_diversity_dict[x])

In [49]:
answers_df

,cbg,safegraph_place_id,Number_of_Visits_2018,Population_in_this_cbg,Median_Income_in_this_cbg,Population_where_store_is,Median_Income_where_store_is,Area_of_store,Distance_between_cbg_and_store,Includes_parking_lot,POI_count_where_store_is,POI_diversity_where_store_is
0,360810140001,sg:51c8d943ff72400ab86e7e8896f4b5af,0,2187,69750.0,287,36006.0,135702,9.780264,0.0000,72,3.083034
1,360810140001,sg:f189f91419974d03a20257132d73da58,12,2187,69750.0,1348,80663.0,149385,4.111893,1.0000,69,2.781085
2,360810140001,sg:c7222887b3834a58a6bc40c168cdb38a,0,2187,69750.0,1280,0.0,3678,8.292913,0.0000,6,1.560710
3,360810140001,sg:e2543bebe82742a1941c21660d9f4168,0,2187,69750.0,1796,75547.0,30554,8.162305,0.0000,39,2.724921
4,360810140001,sg:3275834c58d241baacdb9965f5003af6,0,2187,69750.0,163,0.0,6135,8.393494,1.0000,39,2.929817
...,...,...,...,...,...,...,...,...,...,...,...,...
1844007,360470286004,sg:29b14793c6a54fe69540bddefc24042e,0,1147,59306.0,1823,26917.0,2237,18.623994,1.0000,1,0.000000
1844008,360470286004,sg:187d4b6602c845c3bfede20a1deeb8d1,0,1147,59306.0,1464,30817.0,13671,6.841547,0.0001,6,1.560710
1844009,360470286004,sg:90a04f3d29784b84ad73e55909e864f8,0,1147,59306.0,1348,80663.0,25223,7.679363,0.0001,69,2.781085
1844010,360470286004,sg:10efeb7c06ac4940a3d4e996c0ce5476,0,1147,59306.0,1643,29944.0,28716,16.034442,0.0001,23,2.487258


# M

In [50]:
census_similarity = census_gdf[['census_block_group', 'median_income']]

In [51]:
degree_cols = ['census_block_group']
for i in range(2, 26):
    degree_cols.append('B15003e'+str(i))

In [52]:
race = pd.read_csv(r'F:\safegraph\census data\safegraph_open_census_data\data\cbg_b02.csv'
                  )[['census_block_group','B02001e2', 'B02001e3', 'B02001e4', 'B02001e5', 'B02001e6', 'B02001e7']]
median_age = pd.read_csv(r'F:\safegraph\census data\safegraph_open_census_data\data\cbg_b01.csv'
                  )[['census_block_group','B01002e1']]
education = pd.read_csv(r'F:\safegraph\census data\safegraph_open_census_data\data\cbg_b15.csv'
                  )[degree_cols]

In [53]:
for df_temp in [race, median_age, education]:
    census_similarity = pd.merge(census_similarity, df_temp, on = 'census_block_group')

In [54]:
census_similarity = census_similarity.fillna(0).rename(columns = {'census_block_group': 'cbg'})

In [55]:
census_similarity_compute = census_similarity.drop(columns = 'cbg')

In [56]:
similarities = pd.DataFrame(cosine_similarity(census_similarity_compute, census_similarity_compute))

In [57]:
cbgs_in_queue = census_similarity.cbg.tolist()
def get_cos_sim(cbg_1, cbg_2):
    index_1 = cbgs_in_queue.index(cbg_1)
    index_2 = cbgs_in_queue.index(cbg_2)
    return similarities.iloc[index_1, index_2]

In [58]:
store_cbg_dict = {}
for i, cont in geo_area_stores_gdf.iterrows():
    store_cbg_dict[cont.safegraph_place_id] = cont.cbg

In [59]:
answers_df['Demographic_similarity'] = answers_df.apply(lambda row: get_cos_sim(row.cbg, store_cbg_dict[row.safegraph_place_id]), axis=1)

In [60]:
answers_df

,cbg,safegraph_place_id,Number_of_Visits_2018,Population_in_this_cbg,Median_Income_in_this_cbg,Population_where_store_is,Median_Income_where_store_is,Area_of_store,Distance_between_cbg_and_store,Includes_parking_lot,POI_count_where_store_is,POI_diversity_where_store_is,Demographic_similarity
0,360810140001,sg:51c8d943ff72400ab86e7e8896f4b5af,0,2187,69750.0,287,36006.0,135702,9.780264,0.0000,72,3.083034,0.999842
1,360810140001,sg:f189f91419974d03a20257132d73da58,12,2187,69750.0,1348,80663.0,149385,4.111893,1.0000,69,2.781085,0.999792
2,360810140001,sg:c7222887b3834a58a6bc40c168cdb38a,0,2187,69750.0,1280,0.0,3678,8.292913,0.0000,6,1.560710,0.016467
3,360810140001,sg:e2543bebe82742a1941c21660d9f4168,0,2187,69750.0,1796,75547.0,30554,8.162305,0.0000,39,2.724921,0.999968
4,360810140001,sg:3275834c58d241baacdb9965f5003af6,0,2187,69750.0,163,0.0,6135,8.393494,1.0000,39,2.929817,0.017327
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1844007,360470286004,sg:29b14793c6a54fe69540bddefc24042e,0,1147,59306.0,1823,26917.0,2237,18.623994,1.0000,1,0.000000,0.999232
1844008,360470286004,sg:187d4b6602c845c3bfede20a1deeb8d1,0,1147,59306.0,1464,30817.0,13671,6.841547,0.0001,6,1.560710,0.999131
1844009,360470286004,sg:90a04f3d29784b84ad73e55909e864f8,0,1147,59306.0,1348,80663.0,25223,7.679363,0.0001,69,2.781085,0.999876
1844010,360470286004,sg:10efeb7c06ac4940a3d4e996c0ce5476,0,1147,59306.0,1643,29944.0,28716,16.034442,0.0001,23,2.487258,0.999594


# P and Q

In [61]:
Target_brand = parking_2.copy()
competitors_brand = pd.read_csv(r'F:\safegraph\competitors_1030\places_info_nyc.csv')

In [62]:
brands_dict = {}
for i,cont in competitors_brand.iterrows():
    if cont.safegraph_place_id in final_stores:
        brands_dict[cont.safegraph_place_id] = cont.brands
for i,cont in Target_brand.iterrows():
    if cont.safegraph_place_id in final_stores:
        brands_dict[cont.safegraph_place_id] = cont.brands

In [63]:
answers_df['Brand_name'] = answers_df['safegraph_place_id'].apply(lambda x: brands_dict[x])

In [64]:
brands_id = pd.read_csv(r'F:\safegraph\Core-USA-August2020-Release-CORE_POI-2020_07-2020-08-07\brand_info.csv')

In [65]:
answers_df = pd.merge(answers_df, brands_id[['safegraph_brand_id', 'brand_name']], left_on = 'Brand_name', 
             right_on = 'brand_name').drop(columns = 'brand_name')

In [66]:
answers_df

,cbg,safegraph_place_id,Number_of_Visits_2018,Population_in_this_cbg,Median_Income_in_this_cbg,Population_where_store_is,Median_Income_where_store_is,Area_of_store,Distance_between_cbg_and_store,Includes_parking_lot,POI_count_where_store_is,POI_diversity_where_store_is,Demographic_similarity,Brand_name,safegraph_brand_id
0,360810140001,sg:51c8d943ff72400ab86e7e8896f4b5af,0,2187,69750.0,287,36006.0,135702,9.780264,0.0,72,3.083034,0.999842,Target,SG_BRAND_42aefbae01d2dfd981f7da7d823d689e
1,360810140001,sg:f189f91419974d03a20257132d73da58,12,2187,69750.0,1348,80663.0,149385,4.111893,1.0,69,2.781085,0.999792,Target,SG_BRAND_42aefbae01d2dfd981f7da7d823d689e
2,360810140001,sg:c7222887b3834a58a6bc40c168cdb38a,0,2187,69750.0,1280,0.0,3678,8.292913,0.0,6,1.560710,0.016467,Target,SG_BRAND_42aefbae01d2dfd981f7da7d823d689e
3,360810140001,sg:e2543bebe82742a1941c21660d9f4168,0,2187,69750.0,1796,75547.0,30554,8.162305,0.0,39,2.724921,0.999968,Target,SG_BRAND_42aefbae01d2dfd981f7da7d823d689e
4,360810140001,sg:3275834c58d241baacdb9965f5003af6,0,2187,69750.0,163,0.0,6135,8.393494,1.0,39,2.929817,0.017327,Target,SG_BRAND_42aefbae01d2dfd981f7da7d823d689e
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1831021,360470271002,sg:a15937b5adf141d891dd06a9c81347f6,0,980,50500.0,168,165083.0,19669,5.949294,1.0,411,3.160254,0.999848,Saks Fifth Avenue,SG_BRAND_c8bb13bf059b36b833c4b52648d0ee5a
1831022,360470280002,sg:a15937b5adf141d891dd06a9c81347f6,0,865,58500.0,168,165083.0,19669,10.799933,1.0,411,3.160254,0.999955,Saks Fifth Avenue,SG_BRAND_c8bb13bf059b36b833c4b52648d0ee5a
1831023,360470282002,sg:a15937b5adf141d891dd06a9c81347f6,0,1322,42019.0,168,165083.0,19669,10.856128,1.0,411,3.160254,0.999743,Saks Fifth Avenue,SG_BRAND_c8bb13bf059b36b833c4b52648d0ee5a
1831024,360470282003,sg:a15937b5adf141d891dd06a9c81347f6,0,533,0.0,168,165083.0,19669,10.890959,1.0,411,3.160254,0.000913,Saks Fifth Avenue,SG_BRAND_c8bb13bf059b36b833c4b52648d0ee5a


# R and S

In [67]:
visits_by_brand = answers_df.groupby(['cbg', 'Brand_name'])['Number_of_Visits_'+str(year)].sum().reset_index().rename(
    columns = {'Number_of_Visits_'+str(year): 'Number_of_Visits_by_brand_'+str(year)})

In [68]:
counts_by_brand_df = visits_by_brand.groupby('cbg').sum().reset_index()
counts_by_brand_dict = {}
for i,cont in counts_by_brand_df.iterrows():
    counts_by_brand_dict[cont.cbg] = cont.Number_of_Visits_by_brand_2018

In [69]:
def compute_fraction_brand(row):
    total = counts_by_brand_dict[row.cbg]
    here = row['Number_of_Visits_by_brand_'+str(year)]
    if here == 0:
        return 0
    return here/total

In [70]:
visits_by_brand['Percentage_of_Visits_by_brand_'+str(year)] = visits_by_brand.apply(lambda row: compute_fraction_brand(row), axis=1)

In [71]:
answers_df = pd.merge(answers_df, visits_by_brand, on = ['cbg', 'Brand_name'])

In [72]:
answers_df

,cbg,safegraph_place_id,Number_of_Visits_2018,Population_in_this_cbg,Median_Income_in_this_cbg,Population_where_store_is,Median_Income_where_store_is,Area_of_store,Distance_between_cbg_and_store,Includes_parking_lot,POI_count_where_store_is,POI_diversity_where_store_is,Demographic_similarity,Brand_name,safegraph_brand_id,Number_of_Visits_by_brand_2018,Percentage_of_Visits_by_brand_2018
0,360810140001,sg:51c8d943ff72400ab86e7e8896f4b5af,0,2187,69750.0,287,36006.0,135702,9.780264,0.0,72,3.083034,0.999842,Target,SG_BRAND_42aefbae01d2dfd981f7da7d823d689e,24,0.089552
1,360810140001,sg:f189f91419974d03a20257132d73da58,12,2187,69750.0,1348,80663.0,149385,4.111893,1.0,69,2.781085,0.999792,Target,SG_BRAND_42aefbae01d2dfd981f7da7d823d689e,24,0.089552
2,360810140001,sg:c7222887b3834a58a6bc40c168cdb38a,0,2187,69750.0,1280,0.0,3678,8.292913,0.0,6,1.560710,0.016467,Target,SG_BRAND_42aefbae01d2dfd981f7da7d823d689e,24,0.089552
3,360810140001,sg:e2543bebe82742a1941c21660d9f4168,0,2187,69750.0,1796,75547.0,30554,8.162305,0.0,39,2.724921,0.999968,Target,SG_BRAND_42aefbae01d2dfd981f7da7d823d689e,24,0.089552
4,360810140001,sg:3275834c58d241baacdb9965f5003af6,0,2187,69750.0,163,0.0,6135,8.393494,1.0,39,2.929817,0.017327,Target,SG_BRAND_42aefbae01d2dfd981f7da7d823d689e,24,0.089552
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1831021,360470271002,sg:a15937b5adf141d891dd06a9c81347f6,0,980,50500.0,168,165083.0,19669,5.949294,1.0,411,3.160254,0.999848,Saks Fifth Avenue,SG_BRAND_c8bb13bf059b36b833c4b52648d0ee5a,0,0.000000
1831022,360470280002,sg:a15937b5adf141d891dd06a9c81347f6,0,865,58500.0,168,165083.0,19669,10.799933,1.0,411,3.160254,0.999955,Saks Fifth Avenue,SG_BRAND_c8bb13bf059b36b833c4b52648d0ee5a,0,0.000000
1831023,360470282002,sg:a15937b5adf141d891dd06a9c81347f6,0,1322,42019.0,168,165083.0,19669,10.856128,1.0,411,3.160254,0.999743,Saks Fifth Avenue,SG_BRAND_c8bb13bf059b36b833c4b52648d0ee5a,0,0.000000
1831024,360470282003,sg:a15937b5adf141d891dd06a9c81347f6,0,533,0.0,168,165083.0,19669,10.890959,1.0,411,3.160254,0.000913,Saks Fifth Avenue,SG_BRAND_c8bb13bf059b36b833c4b52648d0ee5a,0,0.000000


# Percentage

In [75]:
answers_df_more = answers_df.copy()

In [76]:
#answers_df.to_csv(r'F:\safegraph\presentation_1105\table.csv')

In [79]:
def compute_fraction(row, y):
    total = counts_by_brand_dict[row.cbg]
    here = row['Number_of_Visits_'+str(y)]
    if here == 0:
        return 0
    return here/total

In [81]:
answers_df_more['Percentage_of_Visits_2018'] = answers_df_more.apply(lambda row: compute_fraction(row, 2018), axis = 1)

# 2019

In [74]:
year = 2019

In [85]:
target_visitors_2019 = get_stores_foot('Target', year)
competitors_visitors_2019 = get_stores_foot('c', year)
visitors_2019 = pd.merge(target_visitors_2019, competitors_visitors_2019, on='cbg')

In [89]:
visitors_dict_2019 = turn_to_dict_visitors(visitors_2019)
answers_df_more['Number_of_Visits_'+str(year)] = answers_df_more.apply(lambda row: get_visitors_data(visitors_dict_2019, row), axis=1)

In [95]:
visits_by_brand_2019 = answers_df_more.groupby(['cbg', 'Brand_name'])['Number_of_Visits_'+str(year)].sum().reset_index().rename(
    columns = {'Number_of_Visits_'+str(year): 'Number_of_Visits_by_brand_'+str(year)})

In [101]:
counts_by_brand_df_2019 = visits_by_brand_2019.groupby('cbg').sum().reset_index()
counts_by_brand_dict_2019 = {}
for i,cont in counts_by_brand_df_2019.iterrows():
    counts_by_brand_dict_2019[cont.cbg] = cont.Number_of_Visits_by_brand_2019

In [109]:
def compute_fraction_2019(row, y):
    total = counts_by_brand_dict_2019[row.cbg]
    here = row['Number_of_Visits_'+str(y)]
    if here == 0:
        return 0
    return here/total

In [110]:
def compute_fraction_brand_2019(row):
    total = counts_by_brand_dict_2019[row.cbg]
    here = row['Number_of_Visits_by_brand_'+str(year)]
    if here == 0:
        return 0
    return here/total

In [112]:
visits_by_brand_2019['Percentage_of_Visits_by_brand_'+str(year)] = visits_by_brand_2019.apply(lambda row: 
                                                                                              compute_fraction_brand_2019(row), axis=1)

In [128]:
visits_by_brand_2019

,cbg,Brand_name,Number_of_Visits_by_brand_2019,Percentage_of_Visits_by_brand_2019
0,360050001000,Amazon 4-Star,0,0.0
1,360050001000,BJ's Wholesale Club,0,0.0
2,360050001000,Barneys New York,0,0.0
3,360050001000,Bloomingdale's,0,0.0
4,360050001000,Bottega Veneta,0,0.0
...,...,...,...,...
155827,360859901000,Saks Off Fifth,0,0.0
155828,360859901000,Sears,0,0.0
155829,360859901000,Sears Home Services,0,0.0
155830,360859901000,T.J. Maxx,0,0.0


In [120]:
answers_df_more = pd.merge(answers_df_more, visits_by_brand_2019, on = ['cbg', 'Brand_name'])

In [122]:
answers_df_more['Percentage_of_Visits_2019'] = answers_df_more.apply(lambda row: compute_fraction_2019(row, 2019), axis = 1)

In [134]:
answers_df_more.to_csv(r'F:\safegraph\presentation_1105\table_comprehensive.csv')